Applied Data Science Capstone

Segmenting and Clustering Neighborhoods in Toronto

by
Marcos Cifuentes
    
    

INSTALL LIBRARIES beautifulsoup4,lxml

In [15]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [202]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import geocoder

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


Solving environment: ...working... done

# All requested packages already installed.



In [167]:
##  requests.get => retriving  information about /List_of_postal_codes_of_Canada
## source takes <!DOCTYPE html> 
## soup SAVE HTML WITH LABELS
## table_contents SAVE CELL BY CELL AFTER ITERATION


 1. Download Dataset
 From the Wikipage given the List of postal codes of Canada :
 Download the HTML file at the given link : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
 Register the file locally
 Open the file and iterate through HTML elements to extract postal codes using 'BeautifulSoup' library

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
table = soup.find_all('table')[0] 
table
table_contents=[]
for row in table.findAll('td'):
    cell = {}
  #  print("===>",''+row.span.text,row.p.text[:3])
  
    if row.span.text=='Not assigned':        
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = 'Not assigned'
        cell['Neighborhood'] = 'Not assigned'
        table_contents.append(cell)
        pass
    else:
        cell_temp=row.span.text,row.p.text[:3]        
       # print("| "+row.span.text,row.p.text[:3]+ " | " + (row.span.text).split('(')[0] + " | " + ((((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' '))+ " | " + row.p.text[:3])

        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
       # print(cell['PostalCode'])
       # print(cell['Borough'])
        #print(cell['Neighborhood'])
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.shape
df.head()
df[0:10]

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [ ]:
#Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, 
#in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [6]:
print('There are {} uniques postal codes.'.format(len(df['PostalCode'].unique())))

There are 180 uniques postal codes.


Here we will use Nominatim already imported in the beginning of the file

I tryied with Geocoder and Nominatim : no results, the program loops without giving more than 1 coordinates. So let's use the csv file.

In [ ]:
#import geocoder
#for index, row in df.iterrows():
#    print(row['PostalCode'], row['Borough'], row['Neighborhood'])
#    address = row['Borough']+ ', '+row['Neighborhood']
#    geolocator = Nominatim(user_agent="explorer")
#    lat_lng_coords = geolocator.geocode(address)
#    print('=>',lat_lng_coords)    
#    # loop until you get the coordinates
#    while(lat_lng_coords != None):
#        g = geocoder.google('{},'.format(row['PostalCode']))
#        lat_lng_coords = g.latlng
#        print('lat_lng_coords=>',lat_lng_coords)    
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]




In [7]:
#!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data   
print('Postal Code Coordinates downloaded!')

df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')


Postal Code Coordinates downloaded!


In [8]:
# Rename the Postal Code column to allow merging
df_coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
print(df_coordinates[0:3])
# for each postal code get the latitude and longitude values
# Merge the 2 dtaframe on the Postalcode column


# for each postal code get the latitude and longitude values
# Merge the 2 dtaframe on the Postalcode column
df.head(10)
df_latlg = pd.merge(df, df_coordinates, on='PostalCode' )
df_latlg.head(50)


  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [143]:
dfToronto  = df[df['Borough'].str.contains('Toronto',regex=False)]
dfToronto.head()
dfToronto.shape

(39, 3)

In [144]:
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [145]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_latlg['Latitude'], df_latlg['Longitude'], df_latlg['Borough'], df_latlg['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [128]:
dfToronto[0:1]

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [146]:
# "denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
df_latlg_toronto_denc = df_latlg[df_latlg['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_latlg_toronto_denc.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [147]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_latlg_toronto_denc['Latitude'], 
        df_latlg_toronto_denc['Longitude'], 
        df_latlg_toronto_denc['Borough'], 
        df_latlg_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc



### Explore the neighborhood's in our data frame "df_latlg_toronto_denc"

In [149]:
cont=0
LIMITE=10
print(f"The "+str(LIMITE)+"th neighborhood's [ df_latlg_toronto_denc ]")
print(f"*************************************************************************************************************************")
while  cont < LIMITE:
    neighborhood_name = df_latlg_toronto_denc.loc[cont, 'Neighborhood'] + ' - [ ' +str(df_latlg_toronto_denc.loc[cont, 'Latitude'])+ ' , ' +str(df_latlg_toronto_denc.loc[cont, 'Longitude'])+ ' ]' 
    print(f"The "+str(cont+1)+"th neighborhood's name is "+neighborhood_name)
    cont=cont+1
print(f"*************************************************************************************************************************")

The 10th neighborhood's [ df_latlg_toronto_denc ]
*************************************************************************************************************************
The 1th neighborhood's name is Regent Park, Harbourfront - [ 43.6542599 , -79.3606359 ]
The 2th neighborhood's name is Garden District, Ryerson - [ 43.6571618 , -79.37893709999999 ]
The 3th neighborhood's name is St. James Town - [ 43.6514939 , -79.3754179 ]
The 4th neighborhood's name is The Beaches - [ 43.67635739999999 , -79.2930312 ]
The 5th neighborhood's name is Berczy Park - [ 43.644770799999996 , -79.3733064 ]
The 6th neighborhood's name is Central Bay Street - [ 43.6579524 , -79.3873826 ]
The 7th neighborhood's name is Christie - [ 43.669542 , -79.4225637 ]
The 8th neighborhood's name is Richmond, Adelaide, King - [ 43.65057120000001 , -79.3845675 ]
The 9th neighborhood's name is Dufferin, Dovercourt Village - [ 43.66900510000001 , -79.4422593 ]
The 10th neighborhood's name is The Danforth  East - [ 43.68534

In [132]:
df_latlg_toronto_denc.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [168]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_latlg_toronto_denc['Latitude'],df_latlg_toronto_denc['Longitude'],df_latlg_toronto_denc['Borough'],df_latlg_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

### Using KMeans clustering for the clsutering of the neighbourhoods

In [ ]:
k=5
toronto_clustering = df_latlg_toronto_denc.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df_latlg_toronto_denc.insert(0, 'Cluster Labels', kmeans.labels_)

In [170]:
df_latlg_toronto_denc

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,2,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,4,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,3,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [172]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_latlg_toronto_denc['Latitude'], df_latlg_toronto_denc['Longitude'], df_latlg_toronto_denc['Neighborhood'], df_latlg_toronto_denc['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


Get the neighborhood's latitude and longitude values.

Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters

In [150]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# 20210712, 
#the geograpical coordinate of Toronto city are 43.6534817, -79.3839347.
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'INGC4CIZSVHHTY5QLQ5GIS0B3XDBSRYW1GRBUPZ1AZIIGAEV', 
    'N5OGQQGWR3URH1ZMPN4BHPMNHZRLJY1KN25DQAK4NA5PFB2Q', 
    '20210712', 
    43.676357,
    -79.293031, 
    500,
    100)

# get the result to a json file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60efa4a20559cb2edafc921c'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857004500005,
    'lng': -79.28682071453191},
   'sw': {'lat': 43.6718569955, 'lng': -79.29924128546809}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distance': 

In [151]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [152]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-152-853095ed29fe>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [121]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues

In [136]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'INGC4CIZSVHHTY5QLQ5GIS0B3XDBSRYW1GRBUPZ1AZIIGAEV', 
    'N5OGQQGWR3URH1ZMPN4BHPMNHZRLJY1KN25DQAK4NA5PFB2Q', 
    '20210712', 
   lat,
   lng, 
   radius,
   LIMIT)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [138]:
df_latlg_toronto_denc.shape

(39, 5)

In [153]:
toronto_venues = getNearbyVenues(names=df_latlg_toronto_denc['Neighborhood'],
                                   latitudes=df_latlg_toronto_denc['Latitude'],
                                   longitudes=df_latlg_toronto_denc['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi


### Let's check the size of the resulting datafram

In [156]:
print(toronto_venues.shape)
toronto_venues.head()

(15, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Enclave of M4L,43.662744,-79.321558,Rorschach Brewing Co.,43.663483,-79.319824,Brewery
1,Enclave of M4L,43.662744,-79.321558,Leslieville Farmers Market,43.664901,-79.319784,Farmers Market
2,Enclave of M4L,43.662744,-79.321558,The Sidekick,43.664484,-79.325162,Comic Shop
3,Enclave of M4L,43.662744,-79.321558,Queen Margherita Pizza,43.664685,-79.324164,Pizza Place
4,Enclave of M4L,43.662744,-79.321558,Chino Locos,43.664653,-79.325584,Burrito Place


In [157]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Enclave of M4L,15,15,15,15,15,15


In [158]:

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 15 uniques categories.


In [159]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Garden Center,Gym / Fitness Center,Light Rail Station,Park,Pizza Place,Restaurant,Skate Park
0,Enclave of M4L,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Enclave of M4L,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Enclave of M4L,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Enclave of M4L,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Enclave of M4L,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
